## Dependences

In [1]:
!pip install --upgrade google-api-python-client
!pip install bitsandbytes>=0.39.0
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install tiktoken
!pip install torch
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 47.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 8.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-uaf34nd3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-uaf34nd3
  Resolved https://github.com/huggingface/transformers.git to commit 30ed3adf474aaf2972ab56f5624089bc24a6adf3
  Installing bu

In [1]:
!nvidia-smi

Thu Jul 13 13:43:54 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070         Off| 00000000:01:00.0 Off |                  N/A |
|  0%   55C    P8               16W / 220W|     15MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!free

              total        used        free      shared  buff/cache   available
Mem:      181108932    10473132    73337164     5609268    97298636   163382608
Swap:             0           0           0


## Imports

In [1]:
import os
import torch
import time
import datetime
import sys
import traceback
import gc

from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## Loading the model

In [2]:
model_name = 'legendhasit/xgen-7b-8k-inst-8bit'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//10.43.0.1'), PosixPath('443'), PosixPath('tcp')}
  warn(msg)
/workspace/.miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent:

## Inference

In [3]:
# Parameters:
# - DATASET_NAME
# - HEADER
# - PROMPT_TEMPLATE
# - samples in dataset 'input/<dataset_name>'
# - instructions in 'instructions.txt'

def mkdir(folder_path):
    try:
        os.mkdir(folder_path)
    except FileExistsError:
        pass

mkdir('input')
mkdir('intermediate')
mkdir('output')

# Define prompt template
# ==========================================================================================
HEADER = (
    "A chat between a curious human and an artificial intelligence assistant. "
    "The assistant gives helpful, detailed, and polite answers to the human's questions."
)

PROMPT_TEMPLATE = HEADER + """

### Human: {instruction}

{article}

### Assistant:"""

# Load samples from dataset
# ==========================================================================================
DATASET_NAME = 'fredsum'
samples = os.listdir('input/' + DATASET_NAME + '/texts')
n_samples = len(samples)
print('Found', n_samples, 'samples')

# Load instructions
# ==========================================================================================
instruction_file = open('instructions.txt', 'r', encoding='utf-8')
instructions = instruction_file.readlines()
instruction_file.close()
n_instructions = len(instructions)
for i in range(n_instructions):
    instructions[i] = instructions[i].replace('\n', '')
print('Found', n_instructions, 'instructions')

# Inference
# ==========================================================================================
initial_time = time.time()
skipped_samples = 0

print('Starting computation...')

# For each instruction
for instruction_n in range(n_instructions):

    # Read instruction and create prompt
    instruction = instructions[instruction_n]
    
    # For each sample in dataset
    for sample_n in range(n_samples):

        # Estimate completion and time.
        cur_samples = instruction_n * n_samples + sample_n - skipped_samples
        tot_samples = n_instructions * n_samples - skipped_samples
        progress = cur_samples / tot_samples
        pct = round(progress * 100, 1)
        print('Prompting instruction N' + str(instruction_n + 1) + '/' + str(n_instructions) + ' on sample N' + str(sample_n + 1) + '/' + str(n_samples))
        print('-- Completion: ' + str(pct) + '%')
        if cur_samples > 0:
            approx_total = (time.time() - initial_time) / cur_samples * tot_samples
            approx_remaining = approx_total * (1 - progress)
            print('-- Estimated Remaining Time: ' + str(datetime.timedelta(seconds=int(approx_remaining))) + ' (total ' + str(datetime.timedelta(seconds=int(approx_total))) + ')')
        
        # Read sample and generate prompt
        sample_file_path = 'input/' + DATASET_NAME + '/texts/' + samples[sample_n]
        sample_file = open(sample_file_path, 'r', encoding='utf-8')
        sample = sample_file.read()
        sample_file.close()
        prompt = PROMPT_TEMPLATE.format(instruction=instruction, article=sample)
        
        # Find target file
        target_file_path = 'intermediate/' + str(instruction_n + 1) + '_' + str(sample_n + 1) + '.txt'
        if os.path.isfile(target_file_path):
            print('-- Found intermediate result file \'' + target_file_path + '\', skipped.')
            skipped_samples += 1
            continue
            
        try:
        
            # Sample one answer
            input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
            sample = model.generate(**input_ids, do_sample=True, max_new_tokens=2048, top_k=100, eos_token_id=50256, temperature=0.3)
            output = tokenizer.decode(sample[0]).strip()

            # Save answer in file
            target_file = open(target_file_path, 'w', encoding='utf-8')
            target_file.write(output)
            target_file.close()

            del input_ids
            del sample
        
        except:
            print('Could not compute prompt:')
            print(prompt)
            traceback.print_exc()

        gc.collect()

delta = time.time() - initial_time
print('Done! Took', datetime.timedelta(seconds=int(delta)), 'seconds')

Found 10 samples
Found 6 instructions
Starting computation...
Prompting instruction N1/6 on sample N1/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_1.txt', skipped.
Prompting instruction N1/6 on sample N2/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_2.txt', skipped.
Prompting instruction N1/6 on sample N3/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_3.txt', skipped.
Prompting instruction N1/6 on sample N4/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_4.txt', skipped.
Prompting instruction N1/6 on sample N5/10
-- Completion: 0.0%
-- Found intermediate result file 'intermediate/1_5.txt', skipped.
Prompting instruction N1/6 on sample N6/10
-- Completion: 0.0%
Prompting instruction N1/6 on sample N7/10
-- Completion: 1.8%
-- Estimated Remaining Time: 1:01:39 (total 1:02:48)
Prompting instruction N1/6 on sample N8/10
-- Completion: 3.6%
-- Estimated Remaining Time: 0:55:54 (total 0:

Traceback (most recent call last):
  File "/tmp/ipykernel_3502/3316153934.py", line 96, in <module>
    sample = model.generate(**input_ids, do_sample=True, max_new_tokens=2048, top_k=100, eos_token_id=50256, temperature=0.3)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py", line 1588, in generate
    return self.sample(
  File "/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py", line 2642, in sample
    outputs = self(
  File "/workspace/.miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/accelerate/hooks.py", line 165, in new_forward
    output = old_forward(*args, **kwargs)
  File "/workspace/.miniconda3/lib/pytho

Prompting instruction N2/6 on sample N2/10
-- Completion: 10.9%
-- Estimated Remaining Time: 0:54:52 (total 1:01:36)
Prompting instruction N2/6 on sample N3/10
-- Completion: 12.7%
-- Estimated Remaining Time: 0:49:37 (total 0:56:51)
Prompting instruction N2/6 on sample N4/10
-- Completion: 14.5%
-- Estimated Remaining Time: 1:10:51 (total 1:22:54)
Prompting instruction N2/6 on sample N5/10
-- Completion: 16.4%
-- Estimated Remaining Time: 1:03:48 (total 1:16:17)


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Prompting instruction N2/6 on sample N6/10
-- Completion: 18.2%
-- Estimated Remaining Time: 1:19:40 (total 1:37:22)
Prompting instruction N2/6 on sample N7/10
-- Completion: 20.0%
-- Estimated Remaining Time: 1:11:32 (total 1:29:26)
Prompting instruction N2/6 on sample N8/10
-- Completion: 21.8%
-- Estimated Remaining Time: 1:22:43 (total 1:45:48)
Prompting instruction N2/6 on sample N9/10
-- Completion: 23.6%
-- Estimated Remaining Time: 1:15:58 (total 1:39:29)
Prompting instruction N2/6 on sample N10/10
-- Completion: 25.5%
-- Estimated Remaining Time: 1:09:35 (total 1:33:21)
Could not compute prompt:
A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: Summarize the following dialogue.

P1 : On avance, on passe à la deuxième grande partie de ce débat, savoir comment vont se dérouler les semaines à venir.

MP : Voilà l'article.

P1 : On va essayer de regarder ça d'ici 

...

Traceback (most recent call last):
  File "/tmp/ipykernel_3502/3316153934.py", line 96, in <module>
    sample = model.generate(**input_ids, do_sample=True, max_new_tokens=2048, top_k=100, eos_token_id=50256, temperature=0.3)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py", line 1588, in generate
    return self.sample(
  File "/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py", line 2642, in sample
    outputs = self(
  File "/workspace/.miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/accelerate/hooks.py", line 165, in new_forward
    output = old_forward(*args, **kwargs)
  File "/workspace/.miniconda3/lib/pytho

Prompting instruction N5/6 on sample N2/10
-- Completion: 65.5%
-- Estimated Remaining Time: 0:22:52 (total 1:06:13)
Prompting instruction N5/6 on sample N3/10
-- Completion: 67.3%
-- Estimated Remaining Time: 0:21:16 (total 1:05:01)
Prompting instruction N5/6 on sample N4/10
-- Completion: 69.1%
-- Estimated Remaining Time: 0:19:55 (total 1:04:28)
Prompting instruction N5/6 on sample N5/10
-- Completion: 70.9%
-- Estimated Remaining Time: 0:18:24 (total 1:03:15)
Prompting instruction N5/6 on sample N6/10
-- Completion: 72.7%
-- Estimated Remaining Time: 0:18:47 (total 1:08:52)
Prompting instruction N5/6 on sample N7/10
-- Completion: 74.5%
-- Estimated Remaining Time: 0:17:10 (total 1:07:28)
Prompting instruction N5/6 on sample N8/10
-- Completion: 76.4%
-- Estimated Remaining Time: 0:15:39 (total 1:06:14)
Prompting instruction N5/6 on sample N9/10
-- Completion: 78.2%
-- Estimated Remaining Time: 0:14:50 (total 1:08:03)
Prompting instruction N5/6 on sample N10/10
-- Completion: 80.0%

Traceback (most recent call last):
  File "/tmp/ipykernel_3502/3316153934.py", line 96, in <module>
    sample = model.generate(**input_ids, do_sample=True, max_new_tokens=2048, top_k=100, eos_token_id=50256, temperature=0.3)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py", line 1588, in generate
    return self.sample(
  File "/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py", line 2642, in sample
    outputs = self(
  File "/workspace/.miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/accelerate/hooks.py", line 165, in new_forward
    output = old_forward(*args, **kwargs)
  File "/workspace/.miniconda3/lib/pytho

Prompting instruction N6/6 on sample N2/10
-- Completion: 83.6%
-- Estimated Remaining Time: 0:11:51 (total 1:12:28)
Prompting instruction N6/6 on sample N3/10
-- Completion: 85.5%
-- Estimated Remaining Time: 0:10:22 (total 1:11:22)
Prompting instruction N6/6 on sample N4/10
-- Completion: 87.3%
-- Estimated Remaining Time: 0:08:55 (total 1:10:03)
Prompting instruction N6/6 on sample N5/10
-- Completion: 89.1%
-- Estimated Remaining Time: 0:07:31 (total 1:08:55)
Prompting instruction N6/6 on sample N6/10
-- Completion: 90.9%
-- Estimated Remaining Time: 0:06:39 (total 1:13:15)
Prompting instruction N6/6 on sample N7/10
-- Completion: 92.7%
-- Estimated Remaining Time: 0:05:36 (total 1:17:09)
Prompting instruction N6/6 on sample N8/10
-- Completion: 94.5%
-- Estimated Remaining Time: 0:04:25 (total 1:21:11)
Prompting instruction N6/6 on sample N9/10
-- Completion: 96.4%
-- Estimated Remaining Time: 0:02:58 (total 1:21:52)
Prompting instruction N6/6 on sample N10/10
-- Completion: 98.2%

Traceback (most recent call last):
  File "/tmp/ipykernel_3502/3316153934.py", line 96, in <module>
    sample = model.generate(**input_ids, do_sample=True, max_new_tokens=2048, top_k=100, eos_token_id=50256, temperature=0.3)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py", line 1588, in generate
    return self.sample(
  File "/workspace/.miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py", line 2642, in sample
    outputs = self(
  File "/workspace/.miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/workspace/.miniconda3/lib/python3.10/site-packages/accelerate/hooks.py", line 165, in new_forward
    output = old_forward(*args, **kwargs)
  File "/workspace/.miniconda3/lib/pytho

In [9]:
del input_ids
del sample
del output

## Calculs de scores

In [ ]:
!pip install rouge
!pip install evaluate
!pip install bert-score
!pip install sacrebleu

In [ ]:
import evaluate
import numpy as np
from rouge import Rouge
from bert_score import score
from sacrebleu.metrics import BLEU, CHRF, TER

In [ ]:
print('Starting scores computation...')
bleu = evaluate.load("bleu")

storage_file = open('output/scores.csv', 'w', encoding='utf-8')

# For each instruction
for instruction_n in range(n_instructions):

    # Read instruction and create prompt
    instruction = instructions[instruction_n]
    
    # For each sample in dataset
    for sample_n in range(n_samples):
        
        # Read sample and generate prompt -> Keep summary 
        summary_file_path = 'input/' + DATASET_NAME + '/summaries/' + samples[sample_n]
        summary_file = open(summary_file_path, 'r', encoding='utf-8')
        summary = summary_file.read()
        summary_file.close()
        
        # Find target file
        compute_hash = hash(instruction + sample)
        target_file_path = 'intermediate/' + str(instruction_hash) + '.txt'
        if not os.path.isfile(target_file_path): # A MODIFIER : SI UN RESUME N'A PAS ETE GENERE
            print('-- Did not Find intermediate result file \'' + target_file_path + '\', skipped.')
            continue

        # Access generated summary
        target_file = open(target_file_path, 'r', encoding='utf-8')
        output = target_file.read()
        target_file.close()

        answers = output.split("### Assistant:") # indice 0 : header + instruction + texte, indices suivants : réponses successives de l'assistant
        answer = ""
        for a in range(1, len(answers)):
            replica_brut = answers[a]
            replica = answerbrut[:-14] # Retirer le token end of text et le \n à la fin.
            answer+=replica

        # Compare summary and answer
        ligne = compute_hash + ";" + compare(summary, answer)
        storage_file.write(ligne + '\n') # REMARQUE SUR LE WRITE ?

print('Done!')


def compare(sum, ans):
    ligne = ""
    result_bleu = bleu.compute(predictions=[ans], references=[[sum]])
    result_rouge = rouge.get_scores(ans, sum)
    P_bert, R_bert, F1_bert = score(ans, sum, lang='en', verbose=True)# Bcp plus d'options possibles
    ligne += result_rouge[0]['rouge-2']['r'] + ";" + result_rouge[0]['rouge-2']['p'] + ";" + result_rouge[0]['rouge-2']['f'] # Stats recall, precision, f de rouge-2
    ligne += ";" + result_rouge[0]['rouge-l']['r'] + ";" + result_rouge[0]['rouge-l']['p'] + ";" + result_rouge[0]['rouge-l']['f'] # Stats recall, precision, f de rouge-l
    ligne += ";" + result_bleu.score # BLEU score
    ligne += ";" + str(R_bert[0].item()) + ";" + str(P_bert[0].item()) + ";" + str(F1_bert[0].item()) # Stats recall, precision, f de BERTScore
    return ligne